# Setting Up PyTorch and CUDA

In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install pandas nltk transformers[torch]
%pip install accelerate -U
%pip install transformers
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Imports here, no need for structure
import os, sys, json, re, string
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch

c:\Users\pete\OneDrive\Desktop\404_finalproj\torchenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!set PATH=C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.5\bin;%PATH%
!set PATH=C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.5\lib\x64;%PATH%

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [ ]:
cuda_bin_path = r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.5\bin"
if cuda_bin_path not in os.environ["PATH"]:
    os.environ["PATH"] = cuda_bin_path + ";" + os.environ["PATH"]
print("Updated PATH to include:", cuda_bin_path)

Updated PATH to include: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.5\bin


# Data Preprocessing

In [ ]:
data_path = r"C:\Users\pete\OneDrive\Documents\School\WWU\Year 3\3. Spring\CSCI 404\final\Reddit_Post_Generator\reddit_wsb.csv"

df = pd.read_csv(data_path)
df.dropna(inplace=True)
df = df[df['score'] >= 100]
df.drop_duplicates(inplace=True)
df.head()
df

,title,score,id,url,comms_num,created,body,timestamp
6,SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,l6uf6d,https://www.reddit.com/r/wallstreetbets/commen...,53,1.611862e+09,Hedgefund whales are spreading disinfo saying ...,2021-01-28 21:26:27
7,THIS IS THE MOMENT,405,l6ub9l,https://www.reddit.com/r/wallstreetbets/commen...,178,1.611862e+09,Life isn't fair. My mother always told me that...,2021-01-28 21:19:31
10,"We need to keep this movement going, we all ca...",222,l6uao1,https://www.reddit.com/r/wallstreetbets/commen...,70,1.611862e+09,I believe right now is one of those rare oppo...,2021-01-28 21:18:25
13,$GME price has nothing to do with fundamentals...,382,l6u96y,https://www.reddit.com/r/wallstreetbets/commen...,61,1.611861e+09,"Firstly, all of you diamond hands and smooth-b...",2021-01-28 21:15:58
19,"Daily Discussion Thread for January 28, 2021",841,l6u011,https://www.reddit.com/r/wallstreetbets/commen...,5942,1.611860e+09,Your daily trading discussion thread. Please k...,2021-01-28 21:00:15
...,...,...,...,...,...,...,...,...
53177,"$CRSR August 20, 2021 Unusual Options Activity 👀",200,owhamt,https://www.reddit.com/r/wallstreetbets/commen...,115,1.627919e+09,"A)50$ strike\n3,048 Open Interest\n310 Volume ...",2021-08-02 18:49:40
53181,Ten Year Price Prediction for TSLA,156,owfbxp,https://www.reddit.com/r/wallstreetbets/commen...,204,1.627913e+09,"It’s all contingent on them mastering FSD, but...",2021-08-02 17:11:36
53182,What I Learned Investigating SAVA FUD Spreaders,238,owd2pn,https://www.reddit.com/r/wallstreetbets/commen...,87,1.627906e+09,***TLDR: Three bitter scientists partnered up ...,2021-08-02 15:03:27
53183,"Daily Popular Tickers Thread for August 02, 20...",228,owd1a5,https://www.reddit.com/r/wallstreetbets/commen...,1070,1.627906e+09,\nYour daily hype thread. Please keep the shit...,2021-08-02 15:01:03


In [ ]:
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

#### Verifying CUDA Works

In [ ]:
print("Compiled with CUDA:", torch.version.cuda)

Compiled with CUDA: 12.1


In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
0
1
NVIDIA GeForce RTX 3080


# Building Model Based on Post Titles

In [ ]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

train_dataset = [tokenizer.encode(title, add_special_tokens=True) for title in df['title']]

In [ ]:
# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    fp16=True,
    logging_steps=500,
    logging_dir='C:/Users/pete/OneDrive/Desktop/404_finalproj/logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

trainer.train()

model.save_pretrained('./fine-tuned-gpt2')
tokenizer.save_pretrained('./fine-tuned-gpt2')


 10%|█         | 501/4809 [00:58<08:28,  8.48it/s]

{'loss': 6.8604, 'grad_norm': 11.436975479125977, 'learning_rate': 4.488459139114161e-05, 'epoch': 0.31}


 21%|██        | 1001/4809 [01:56<07:25,  8.54it/s]

{'loss': 4.4149, 'grad_norm': 8.69334602355957, 'learning_rate': 3.9686005406529424e-05, 'epoch': 0.62}


 31%|███       | 1501/4809 [02:55<06:19,  8.72it/s]

...

 94%|█████████▎| 4501/4809 [08:44<00:36,  8.45it/s]

{'loss': 3.2751, 'grad_norm': 8.937682151794434, 'learning_rate': 3.295903514244126e-06, 'epoch': 2.81}


100%|██████████| 4809/4809 [09:20<00:00,  8.58it/s]


{'train_runtime': 560.5343, 'train_samples_per_second': 34.301, 'train_steps_per_second': 8.579, 'train_loss': 3.9974258495829758, 'epoch': 3.0}


('./fine-tuned-gpt2\\tokenizer_config.json',
 './fine-tuned-gpt2\\special_tokens_map.json',
 './fine-tuned-gpt2\\vocab.json',
 './fine-tuned-gpt2\\merges.txt',
 './fine-tuned-gpt2\\added_tokens.json')

In [ ]:
# Load the fine-tuned model and tokenizer
model_name = './fine-tuned-gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

def generate_post(prompt, max_length=150):
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    generated_post = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_post

# Example prompt
prompt = "Today in the stock market"
generated_post = generate_post(prompt)
print(f"Generated post: {generated_post}")


c:\Users\pete\OneDrive\Desktop\404_finalproj\torchenv\lib\site-packages\transformers\generation\configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated post: Today in the stock market, we have the potential to change the entire market forever. We need to hold and hold. 💎🙌💪🏼🚀🌕🍌.🎥🦍🐻🖐️🤲📉🡌 🚨🪓🧻👐🥽🛽👌👊👋👀👍�🊀✋✊✌✍✎✏✐✔✖✉✒✚✈✪✥✦✝✨✫✮✯✭✡✲✳�


### Grabs random token from post titles, then generates a post based on it

In [ ]:
import random

# Flatten the list of tokenized titles
flat_token_list = [token for sublist in train_dataset for token in sublist]

# Select a random token
random_token = random.choice(flat_token_list)

# Optional: Convert the token back to text to see what it is
random_token_text = tokenizer.decode([random_token])

print("Random Token ID:", random_token)
print("Random Token Text:", random_token_text)

generated_random_post = generate_post(random_token_text)
print(f"Generated post: {generated_random_post}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Random Token ID: 5984
Random Token Text:  AD
Generated post:  ADC - The Big Short is Coming - A Short Squeeze Analysis - Part 2 - DD Part 3 - Why I'm betting on $AMC and $BB to win the day. 🚀🚂💎🙌🏼🌕🦍🍌 💌👐🐻🎯🤲🪓📉🡌.🖐️🔭🨂 🔮🧻👌�✌ ✌☌ ☌️✊✋✍✎✏✐✔✉✈✚✒✖️☊ ✊☎☐☉


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

def setup_summarization_model():
    model_name = 'facebook/bart-large-cnn'  # Pre-trained model for summarization
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)
    return tokenizer, model

In [ ]:
def generate_title(post_body, tokenizer, model, max_length=50):
    inputs = tokenizer.encode("summarize: " + post_body, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=5, length_penalty=2.0, num_beams=4, early_stopping=True)
    post_title = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return post_title

In [ ]:
# Setup the summarization model
summarization_tokenizer, summarization_model = setup_summarization_model()

# Generate a title for the post
post_title = generate_title(generated_random_post, summarization_tokenizer, summarization_model)

In [ ]:
print("Random Token ID:", random_token)
print("Random Token Text:", random_token_text)
print(f"Generated title: {post_title[11:]}")
print(f"Generated post: {generated_random_post}")

Random Token ID: 5984
Random Token Text:  AD
Generated title:  ADC - The Big Short is Coming - A Short Squeeze Analysis - Part 2 - DD Part 3 - Why I'm betting on $AMC and $BB to win the day.
Generated post:  ADC - The Big Short is Coming - A Short Squeeze Analysis - Part 2 - DD Part 3 - Why I'm betting on $AMC and $BB to win the day. 🚀🚂💎🙌🏼🌕🦍🍌 💌👐🐻🎯🤲🪓📉🡌.🖐️🔭🨂 🔮🧻👌�✌ ✌☌ ☌️✊✋✍✎✏✐✔✉✈✚✒✖️☊ ✊☎☐☉


In [ ]:
def normalize_text(text):

    text = text.lower()

    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)


    text = text.translate(str.maketrans("", "", string.punctuation))

    text = re.sub(r'\d+', '', text)

    text = ' '.join(text.split())


    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])

    return text

def count_words(text):
    return len(text.split())

In [ ]:
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

data_path = r"C:\Users\pete\OneDrive\Documents\School\WWU\Year 3\3. Spring\CSCI 404\final\Reddit_Post_Generator\reddit_wsb.csv"

df = pd.read_csv(data_path)
df.dropna(inplace=True)
df = df[df['score'] >= 100]
df.drop_duplicates(inplace=True)
df.head()

tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

df['word_count'] = df['body'].apply(count_words)
df.drop('word_count', axis=1, inplace=True)
df['normal_body'] = df['body'].apply(normalize_text)

# Generating Post Based on Body Content

In [ ]:
max_length = 64  # Define the maximum sequence length for GPT-2

# Encode the dataset with truncation and padding
body_train_dataset = [
    tokenizer.encode(
        body,
        max_length=max_length,
        truncation=True,
        padding="max_length",
        add_special_tokens=True
    ) for body in df['body']
]

### Setting up the Model

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    fp16=True,
    logging_steps=500,
    logging_dir='C:/Users/pete/OneDrive/Desktop/404_finalproj/logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=body_train_dataset,
)

trainer.train()

model.save_pretrained('./body-fine-tuned-gpt2')
tokenizer.save_pretrained('./body-fine-tuned-gpt2')

 10%|█         | 501/4809 [00:57<08:20,  8.61it/s]

{'loss': 5.9946, 'grad_norm': 5.952375888824463, 'learning_rate': 4.488459139114161e-05, 'epoch': 0.31}


 21%|██        | 1001/4809 [01:55<07:19,  8.66it/s]

{'loss': 3.5796, 'grad_norm': 5.326918125152588, 'learning_rate': 3.9686005406529424e-05, 'epoch': 0.62}


 31%|███       | 1501/4809 [02:52<06:19,  8.71it/s]

...

 94%|█████████▎| 4501/4809 [08:31<00:34,  8.83it/s]

{'loss': 2.9558, 'grad_norm': 4.396152973175049, 'learning_rate': 3.295903514244126e-06, 'epoch': 2.81}


100%|██████████| 4809/4809 [09:06<00:00,  8.80it/s]


{'train_runtime': 546.4301, 'train_samples_per_second': 35.187, 'train_steps_per_second': 8.801, 'train_loss': 3.4617446769322493, 'epoch': 3.0}


('./body-fine-tuned-gpt2\\tokenizer_config.json',
 './body-fine-tuned-gpt2\\special_tokens_map.json',
 './body-fine-tuned-gpt2\\vocab.json',
 './body-fine-tuned-gpt2\\merges.txt',
 './body-fine-tuned-gpt2\\added_tokens.json')

In [ ]:
# Load the fine-tuned model and tokenizer
body_model_name = './body-fine-tuned-gpt2'
body_tokenizer = GPT2Tokenizer.from_pretrained(body_model_name)
body_model = GPT2LMHeadModel.from_pretrained(body_model_name)

def body_generate_post(prompt, max_length=150):
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = body_model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    body_generated_post = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return body_generated_post

# Example prompt
prompt = "Today in the stock market"
body_generated_post = body_generate_post(prompt)
print(f"Generated post: {body_generated_post}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated post: Today in the stock market, there are a lot of people who are holding GME. I’m not one of them. But I do have a few shares of GMEs.

I”m holding them because I believe in them and I want to see them go. They are the ones who will make us rich. We‚re all going to have to pay for the GMES. 
  


&#x200B;


  I have been holding these shares for a while now. When I first started holding, I was holding $GME. Now I hold $AMC. And I am holding AMC. So I think it‘s time to buy AMC shares. It


### Building a random post based on the body trained model

In [ ]:
# Flatten the list of tokenized titles
body_flat_token_list = [token for sublist in body_train_dataset for token in sublist]

# Select a random token
body_random_token = random.choice(body_flat_token_list)

# Optional: Convert the token back to text to see what it is
body_random_token_text = body_tokenizer.decode([body_random_token])

print("Random Token ID:", body_random_token)
print("Random Token Text:", body_random_token_text)

body_generated_random_post = body_generate_post(body_random_token_text)
print(f"Generated post: {body_generated_random_post}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Random Token ID: 21767
Random Token Text:  equals
Generated post:  equals the amount of money you have to invest in a stock. 

If you are a billionaire and you own a company, you can buy a share of the company at a discount. If you hold a lot of shares, it is a good idea to buy shares at the discount, because the stock price will rise.
  


The stock market is rigged. The stock is manipulated. It is not a market where you buy and hold shares. You are buying shares for the price you want to pay. This is why the market has been rigged so that the hedge funds and other big players have been able to manipulate the markets to their advantage. They have manipulated the prices of all the other stocks to make the shares more expensive


In [ ]:
body_post_title = generate_title(body_generated_random_post, summarization_tokenizer, summarization_model)

In [ ]:
print("Random Token ID:", body_random_token)
print("Random Token Text:", body_random_token_text)
print(f"Generated title: {body_post_title}")
print(f"Generated post: {body_generated_random_post}")

Random Token ID: 21767
Random Token Text:  equals
Generated title: If you are a billionaire and you own a company, you can buy a share of the company at a discount. If you hold a lot of shares, it is a good idea to buy shares at the discount, because the stock price
Generated post:  equals the amount of money you have to invest in a stock. 

If you are a billionaire and you own a company, you can buy a share of the company at a discount. If you hold a lot of shares, it is a good idea to buy shares at the discount, because the stock price will rise.
  


The stock market is rigged. The stock is manipulated. It is not a market where you buy and hold shares. You are buying shares for the price you want to pay. This is why the market has been rigged so that the hedge funds and other big players have been able to manipulate the markets to their advantage. They have manipulated the prices of all the other stocks to make the shares more expensive
